In [1]:
import numpy as np
from torch import nn, autograd
import torch
from torch.autograd import Variable
from torch.nn import functional as F
from torch import optim

In [2]:
data_path = '../dataset'

In [3]:
def load_data(filename):
    return np.load(data_path+'/'+filename)

In [4]:
vocab = load_data('vocab.npy')
wiki_train = load_data('wiki.train.npy')
wiki_valid = load_data('wiki.valid.npy')

In [11]:
HIDDEN_DIM1 = 10
class model(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_dim):
        super(model, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        
        self.lstm1 = nn.LSTM(embedding_size, hidden_dim)
        
        self.hidden2Lin = nn.Linear(hidden_dim, vocab_size)
        
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda())
        
    def forward(self, sentence):
        sentence_torch = Variable(torch.from_numpy(sentence)).long().cuda()
        embeds = self.embedding(sentence_torch)
        lstm_out, self.hidden = self.lstm1(embeds.view(len(sentence), 1, -1), self.hidden)
        
        fc_out = self.hidden2Lin(lstm_out.view(len(sentence) ,-1))
#         prob =  F.cross_entropy(fc_out, dim=1)
        return fc_out

In [12]:
%%time
mdl = model(vocab_size=len(vocab), embedding_size=HIDDEN_DIM1, hidden_dim=1000)
mdl.cuda()
loss_func = nn.CrossEntropyLoss()
opt = optim.SGD(mdl.parameters(), lr=0.1)

Wall time: 631 ms


In [16]:
for ep in range(10):
    for sentence in wiki_train:
        mdl.zero_grad()
        
        mdl.init_hidden()
        
        output_score = mdl(sentence.astype(int)) 
        sentence_torch = Variable(torch.from_numpy(sentence)).long().cuda()

        loss = loss_func(output_score, sentence_torch)
        loss.backward()
        print(loss[-1])
        opt.step()

Variable containing:
 10.4103
[torch.cuda.FloatTensor of size 1 (GPU 0)]



RuntimeError: cuda runtime error (2) : out of memory at d:\pytorch\pytorch\torch\lib\thc\generic/THCStorage.cu:58